In [ ]:
"""
Cell For Papermill Parameters
"""

images_to_use = 25
x_crop = 56
y_crop = 56
folder_path = "../../prostate_cancer_data/"

In [ ]:
import pydicom as dicom
import os
import cv2
import pandas as pd
import math
import glob
from pathlib import Path

In [ ]:
def process_folder(folder_path):
    
    """
    process_folder processes DCOM images in the folder
    
    :param folder_path: folder to process
    :return: jpg_list,dcm_values
    """ 
    
    jpg_list = []
    dcm_values = []
    
    images_path = os.listdir(folder_path)

    for image in images_path:
        full_path = os.path.join(folder_path, image)
        if image.endswith(".dcm"):
            ds = dicom.dcmread(full_path)
        
            jpg_image_name = full_path+".jpg"
            jpg_list.append(jpg_image_name)
            cv2.imwrite(jpg_image_name, ds.pixel_array)
                               
            patient_name=''
            patient_age=''
            patient_size=''
            patient_weight=''
            patient_eth=''
            patient_occ=''
            patient_smoke=''
            
            try:
                patient_name = ds[0x0010,0x0010].value
            except:
                pass
            
            try:
                patient_age = ds[0x0010,0x1010].value
            except:
                pass
            
            try:
                patient_size = ds[0x0010,0x1020].value
            except:
                pass

            try:
                patient_weight = ds[0x0010,0x1030].value
            except:
                pass
            
            try:
                patient_eth = ds[0x0010,0x2160].value
            except:
                pass            
            
            try:
                patient_occ = ds[0x0010,0x2180].value
            except:
                pass            
            
            try:
                patient_smoke = ds[0x0010,0x21a0].value
            except:
                pass  
            
            dcm_values.append([patient_name,patient_age,patient_size,patient_weight,patient_eth,patient_occ,patient_smoke])
            
    return jpg_list,dcm_values

In [ ]:
def crop_image(folder_path,x_crop,y_crop):
    
    """
    crop_image crops the image
    
    :param folder_path: folder to process
    :param x_crop: pixel crop x
    :param y_crop: pixel crop y
    """ 
    
    fileList = glob.glob(folder_path + '*jpg*')
    
    # Iterate over images
    for filePath in fileList:
        img = cv2.imread(filePath)
        image_shape_x = img.shape[0]
        image_shape_y = img.shape[1]
                        
        crop_img = img[(0+y_crop):(image_shape_y-y_crop), (0+x_crop):(image_shape_x-x_crop)]
        cv2.imwrite(filePath, crop_img)
        

In [ ]:
def remove_jpg(folder_path):
  
    """
    remove_jpg removes jpg from folder
    
    :param folder_path: folder to process
    """ 
    
    # ==========================================================================================
    # Remove old Final Files
    # ==========================================================================================
    fileList1  = glob.glob(folder_path + '**/*AX_T2*/*jpg*' , recursive=True)
    fileList2 = glob.glob(folder_path + '**/*PROPELLER*/*jpg*' , recursive=True)
    fileList3 = glob.glob(folder_path + '**/*axial*/*jpg*' , recursive=True)
    fileList4 = glob.glob(folder_path + '**/*T2 Ax*/*jpg*' , recursive=True)
    fileList5 = glob.glob(folder_path + '**/*AX T2*/*jpg*' , recursive=True)
    fileList6 = glob.glob(folder_path + '**/*AX */*jpg*' , recursive=True)
    fileList7 = glob.glob(folder_path + '**/*T2 AX*/*jpg*' , recursive=True)
    fileList8 = glob.glob(folder_path + '**/*T2 SPACE*/*jpg*' , recursive=True)
    fileList = fileList1 + fileList2 + fileList3 + fileList4 + fileList5 + fileList6 + fileList7 + fileList8  
        
    # Iterate over the list of filepaths & remove each file.
    for filePath in fileList:    
        try:            
            os.remove(filePath)
        except:
            pass

In [ ]:
def make_patient_image(folder_path,images_to_use):
    
    """
    make_patient_image creates images
    
    :param folder_path: folder to process
    :param images_to_use: images to use
    """ 
               
    # ==========================================================================================
    # List all Images
    # ==========================================================================================
    images_path = os.listdir(folder_path)
    image_list = []
        
    for image in images_path:
        full_path = os.path.join(folder_path, image)
        if image.endswith(".jpg"):
            image_list.append(full_path)            
    
    #sort images
    image_list.sort()
    
    image_cnt = len(image_list)
    
    image_ignore = math.floor((image_cnt- images_to_use)/2)
            
    #remove first x images
    image_list = image_list[image_ignore:]
    
    #remove last x images
    image_list = image_list[:len(image_list)-image_ignore]
    
    image_list = image_list[:images_to_use]
            
    image_cnt = len(image_list)
    print(image_cnt)
    
    if image_cnt == 25:
        dims = 5
    elif image_cnt == 16:
        dims = 4
    elif image_cnt == 4:
        dims = 2
    elif image_cnt < 4:
        dims = 1        
    else:
        dims = math.ceil((math.sqrt(image_cnt)))   
                              
    # ==========================================================================================
    # Looping Over Images to Concatenate and create Vertical
    # ==========================================================================================
    for coll_cnt in range(0,dims):

        final_image = os.path.join(folder_path, "finalV" + str(coll_cnt)+".jpg")
        zero_flag = 0
        
        #print("======================================")
        
        for x in range(1,(dims)):        

            image_number = (x - 1) + (coll_cnt * dims)

            if image_number >= (image_cnt-1):
                # set image file names
                img1 = final_image
                img2 = image_list[(0)]

            elif zero_flag == 0:
                # set image file names
                img1 = image_list[image_number]
                img2 = image_list[(image_number+1)]            
                zero_flag = 1
            else:
                # set image file names
                img1 = final_image
                img2 = image_list[(image_number+1)]
                
            # read the images           
            img1 = cv2.imread(img1)
            img2 = cv2.imread(img2)
            
            try:
                im_v = cv2.vconcat([img1, img2])   
                cv2.imwrite(final_image, im_v)        
            except:
                pass
                        
    # ==========================================================================================
    # Looping Over finalV images to concatenate into final image
    # ==========================================================================================    
    
    final_image = os.path.join(folder_path, "final.jpg")
    image_list = glob.glob(folder_path + '*final*')
    image_cnt = len(image_list) - 1
    zero_flag = 0
    image_list.sort()
        
    for x in range(0,image_cnt):

        if zero_flag == 0:
            # set image file names
            img1 = image_list[x]
            img2 = image_list[(x+1)]            
            zero_flag = 1
        else:
            # set image file names
            img1 = final_image
            img2 = image_list[(x+1)]
            
        # read the images
        img1 = cv2.imread(img1)
        img2 = cv2.imread(img2)

        im_v = cv2.hconcat([img1, img2])   
        cv2.imwrite(final_image, im_v)        

In [ ]:
fileList = glob.glob(folder_path + '**/*obl_Prostate*' , recursive=True)
dcm_metadata = []

print(len(fileList))
counter = 0

# Remove all JPG
remove_jpg(folder_path)

# Sort List
fileList.sort()

for folder in fileList:
    print("=====================================================================================")
    print(counter)
    print(folder)
            
    # Convert dicom to JPEG
    jpg_list, dcm_values = process_folder(folder+"/")
    
    # Append Metadata to FinalList
    dcm_metadata.append(dcm_values)
    
    # Crop Image
    crop_image(folder+"/",x_crop,y_crop)
    
    # Make collage
    make_patient_image(folder+"/",images_to_use)
    
    counter = counter + 1

In [ ]:
# Create Empty DataFrame
df = pd.DataFrame(columns=['name','age','size','weight','ethnic_grp','occupation','smoking_status'])

# Get Patient List
counter = 0
for entry in dcm_metadata:
    df.loc[counter] = [entry[0][0]] + [entry[0][1]] + [entry[0][2]] + [entry[0][3]] + [entry[0][4]] + [entry[0][5]] + [entry[0][6]]
    counter = counter + 1

df.to_csv("patients.csv", sep=',', encoding='utf-8', index=False)